<a href="https://colab.research.google.com/github/SEUNGTO/botdata/blob/main/ETF_changes_notice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# 0. 기초 세팅
- 사용방법 : 아래 셀들을 모두 실행

In [ ]:
import io
import time
import json
import requests
import pandas as pd
from bs4 import BeautifulSoup
from tqdm import tqdm

In [ ]:
def codeListing() :

    otp_url = 'http://data.krx.co.kr/comm/fileDn/GenerateOTP/generate.cmd'
    otp_params = {
        'locale': 'ko_KR',
        'share': '1',
        'csvxls_isNo': 'false',
        'name': 'fileDown',
        'url': 'dbms/MDC/STAT/standard/MDCSTAT04601'
    }
    headers = {'Referer' : 'http://data.krx.co.kr/contents/MDC/MDI/mdiLoader'}
    otp = requests.post(otp_url, params = otp_params, headers = headers).text

    down_url = 'http://data.krx.co.kr/comm/fileDn/download_csv/download.cmd'
    down_params = {'code' : otp}
    response = requests.post(down_url, params = down_params, headers = headers)

    data = pd.read_csv(io.BytesIO(response.content), encoding = 'euc-kr', dtype = {'단축코드': 'string'})

    return data

In [ ]:
def PDFListing(isuCd, code, name, date) :

    headers = {'Referer' : 'http://data.krx.co.kr/contents/MDC/MDI/mdiLoader'}

    otp_url = 'http://data.krx.co.kr/comm/fileDn/GenerateOTP/generate.cmd'
    otp_params = {
        'locale': 'ko_KR',
        'tboxisuCd_finder_secuprodisu1_0' : f'{code}/{name}',
        'isuCd': f'{isuCd}',
        'isuCd2': f'{isuCd}',
        'codeNmisuCd_finder_secuprodisu1_0': f'{name}',
        'param1isuCd_finder_secuprodisu1_0': "",
        'trdDd': f'{date}',
        'share': '1',
        'money': '1',
        'csvxls_isNo': 'false',
        'name': 'fileDown',
        'url': 'dbms/MDC/STAT/standard/MDCSTAT05001'
        }

    otp = requests.post(otp_url, params = otp_params, headers = headers).text

    down_url = 'http://data.krx.co.kr/comm/fileDn/download_csv/download.cmd'
    down_params = {'code' : otp}
    response = requests.post(down_url, params = down_params, headers = headers)

    data = pd.read_csv(io.BytesIO(response.content), encoding = 'euc-kr')

    return data

In [ ]:
def loadData() :

    url = 'https://raw.githubusercontent.com/SEUNGTO/botdata/main/resultDict.json'
    response = requests.get(url)
    resultDict = response.json()

    for code in resultDict.keys() :
        for date in resultDict[code].keys() :
            resultDict[code][date] = pd.DataFrame(resultDict[code][date])

    return resultDict

# 1. 데이터 쌓기
사용방법
- 적재하고자 하는 날짜 지정(yyyymmdd형식)
- 아래 셀을 실행

In [ ]:
# 코드 불러오기
codeList = codeListing()
codeList = codeList[(codeList['기초시장분류'] == '국내') & (codeList['기초자산분류'] == '주식')]

In [ ]:
# 기존 데이터 불러오기
resultDict = loadData()

In [ ]:
# 데이터 쌓기
date = '20240412'

for isuCd, code, name in tqdm(zip(codeList['표준코드'], codeList['단축코드'], codeList['한글종목약명'])) :
    data = PDFListing(isuCd, code, name, date)
    if code not in resultDict.keys() :
        tmpDict = {}
        tmpDict[date] = data
        resultDict[code] = tmpDict
        time.sleep(0.5)
    else :
        resultDict[code][date] = data
        time.sleep(0.5)

363it [19:10,  3.17s/it]


In [ ]:
# 종료 및 결과 저장 (json)
from google.colab import files

for code in resultDict.keys() :
    for date in resultDict[code].keys() :
        resultDict[code][date] = resultDict[code][date].to_dict()

with open(f'resultDict.json', 'w') as f :
    json.dump(resultDict, f)

files.download(f'resultDict.json')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# 2. 변동 확인

In [ ]:
def findChange(bas_dt, target_date, codeList, resultDict) :

    for idx, code in enumerate(codeList['단축코드']) :
        if idx == 0 :

            basData = resultDict[code][bas_dt]
            targetData = resultDict[code][target_date]

            chgData = pd.merge(basData[['종목코드', '구성종목명','주식수(계약수)']],
                            targetData[['종목코드', '주식수(계약수)']],
                            how = 'left',
                            on = '종목코드'
                                        )
            chgData['변화분'] = chgData['주식수(계약수)_y'] - chgData['주식수(계약수)_x']
            chgData.fillna(0, inplace = True)
            chgStocks = chgData[chgData['변화분'] != 0][['종목코드', '변화분']]
            chgStocks['ETF'] = code
        else :
            basData = resultDict[code][bas_dt]
            targetData = resultDict[code][target_date]

            tmp = pd.merge(basData[['종목코드', '구성종목명','주식수(계약수)']],
                            targetData[['종목코드', '주식수(계약수)']],
                            how = 'left',
                            on = '종목코드'
                                        )
            tmp['변화분'] = tmp['주식수(계약수)_y'] - tmp['주식수(계약수)_x']
            tmp.fillna(0, inplace = True)
            tmp = tmp[tmp['변화분'] != 0][['종목코드','구성종목명', '변화분']]
            tmp['ETF'] = code

            chgStocks = pd.concat([chgStocks, tmp])

    ETFcodeName = codeList[['단축코드', '한글종목약명']]
    ETFcodeName.columns = ['ETF', 'ETF종목명']
    chgStocks = pd.merge(chgStocks, ETFcodeName, how = 'left', on = 'ETF')
    chgStocks = chgStocks[['ETF', 'ETF종목명', '종목코드', '구성종목명', '변화분']]

    return chgStocks

In [ ]:
# 코드 불러오기
codeList = codeListing()
codeList = codeList[(codeList['기초시장분류'] == '국내') & (codeList['기초자산분류'] == '주식')]

In [ ]:
# 데이터 불러오기
resultDict = loadData()

In [ ]:
# 변동 확인
bas_dt = '20240411'
target_date = '20240412'

chgStocks = findChange(bas_dt, target_date, codeList, resultDict)
chgStocks

,ETF,ETF종목명,종목코드,구성종목명,변화분
0,252420,KBSTAR 200선물인버스2X,KR4101V60002,코스피200 F 202406,-0.01
1,465350,KBSTAR 2차전지TOP10인버스(합성),KRYZTRSE4807,스왑(NH투자증권)_240408,488151.35
2,465350,KBSTAR 2차전지TOP10인버스(합성),KRYZTRSE3T06,스왑(메리츠증권)_44F1_240327,147600.85
3,465350,KBSTAR 2차전지TOP10인버스(합성),KRYZTRSDCB64,스왑(미래에셋증권)_231211,2049854.18
4,465350,KBSTAR 2차전지TOP10인버스(합성),KRYZTRSE3V06,스왑(키움증권)_44F1_240328,746885.49
...,...,...,...,...,...
872,305050,ACE 코스피,138040,메리츠금융지주,-5.00
873,253150,ARIRANG 200선물레버리지,477050,ARIRANG 머니마켓액티브,20.00
874,395760,ARIRANG ESG성장주액티브,034020,두산에너빌리티,-117.00
875,395760,ARIRANG ESG성장주액티브,090430,아모레퍼시픽,17.00


In [ ]:
chgStocks.to_excel('chgPDFs.xlsx', index = False)